In [ ]:
#!pip install mne_qt_browser pyqt5 bsl pylsl liesl

In [ ]:
import time

import bsl
from bsl import StreamPlayer, datasets
from bsl.externals import pylsl  # distributed version of pylsl
from bsl.triggers import TriggerDef

import liesl

import pylsl
import math
import matplotlib
import matplotlib.pyplot as plt

import mne
import numpy as np
import os
# %matplotlib qt
# mne.set_config('MNE_BROWSER_BACKEND', 'qt')

In [ ]:
idun_guardian_client_beta_folder = os.getcwd()

In [ ]:
import pandas as pd

df = pd.read_csv(f'{idun_guardian_client_beta_folder}/eeg.csv')

In [ ]:
info = mne.create_info(sfreq=250, ch_names=['E1'], ch_types=['eeg'])
data = np.array([df['ch1'].values])
raw = mne.io.RawArray(data=data, info=info)

In [ ]:
raw.save(f'{idun_guardian_client_beta_folder}/eeg.fif', overwrite=True)

In [ ]:
raw.plot()

In [ ]:
raw.plot_psd(picks='E1')

In [ ]:
# à augmenter : Alpha, theta
# à diminuer : gamma, beta (, delta)

In [ ]:
psds, freqs = mne.time_frequency.psd_welch(raw, fmin=8, fmax=12, picks='E1')
psds.mean()

In [ ]:
freqs

In [ ]:
psds

### Save raw as fif file

In [ ]:
raw.save("eeg.fif", overwrite=True)

# Timestamps

In [ ]:
df['timestamp'] = df['timestamp'] - df['timestamp'].iloc[0]

In [ ]:
plt.plot(df['timestamp'])